In [15]:
import torch.nn as nn
import torch.nn.functional as F

from torchvision.datasets import FashionMNIST
from torchvision import transforms

import numpy as np

In [3]:
train_dataset = FashionMNIST(root=".", train=True, download=True)
test_dataset = FashionMNIST(root=".", train=False, download=True)

Processing...
Done!


In [10]:
train_dataset[0][1]

tensor(9)

In [66]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv_layer_1 = nn.Conv2d(28, 28, 1, 1)
        #self.pool = nn.MaxPool2d(2, 2)
        self.conv_layer_2 = nn.Conv2d(28, 16, 5)
        
    def forward(self, input_image):
        result_1 = F.relu(self.conv_layer_1(input_image))
        result_2 = F.relu(self.conv_layer_1(result_1))
        return F.softmax(result_2, dim=1)

In [67]:
model = Model()

In [68]:
image = transforms.ToTensor()(train_dataset[0][0])
model.forward(image.view(28, 28, 1, 1))
#pixels = np.asarray(train_dataset[0][0])


RuntimeError: Dimension out of range (expected to be in range of [-4, 3], but got 10)